<a href="https://colab.research.google.com/github/vee234o/promotion-model/blob/main/promotion_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_sample_weight

In [30]:
#loaded the data
df = pd.read_csv('Promotion Dataset (1).csv')
df.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,MSc MBA and PhD,Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [31]:
!pip install ydata_profiling
from ydata_profiling import ProfileReport
ProfileReport(df).to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 19/19 [00:01<00:00, 10.45it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
#dropped irrelevant columns
df1 = df.drop('EmployeeNo', axis = 1)

In [33]:
df1.isna().sum()

,0
Division,0
Qualification,1679
Gender,0
Channel_of_Recruitment,0
Trainings_Attended,0
Year_of_birth,0
Last_performance_score,0
Year_of_recruitment,0
Targets_met,0
Previous_Award,0


In [34]:

#dropped all rows containing missing values in the qualification column
df1[['No_of_previous_employers', 'Promoted_or_Not']].dropna()
df1['Qualification'].fillna(df1['Qualification'].mode()[0])

,Qualification
0,MSc MBA and PhD
1,First Degree or HND
2,First Degree or HND
3,First Degree or HND
4,First Degree or HND
...,...
38307,First Degree or HND
38308,MSc MBA and PhD
38309,First Degree or HND
38310,First Degree or HND


In [35]:
# Correlation only works on numbers, so select them first
num_cols = df.select_dtypes(include=['number'])
print(num_cols.corr()['Promoted_or_Not'].sort_values(ascending=False))

Promoted_or_Not             1.000000
Targets_met                 0.224518
Previous_Award              0.201434
Training_score_average      0.178448
Last_performance_score      0.119690
Year_of_birth               0.017991
Year_of_recruitment         0.012287
No_of_previous_employers    0.001690
Trainings_Attended         -0.024345
Name: Promoted_or_Not, dtype: float64


In [36]:
for col in df1.select_dtypes(include=['object']).columns:
  df1[col] = LabelEncoder().fit_transform(df1[col])

In [37]:
df1.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,1,1,0,1,2,1986,12.5,2011,1,0,41,3,0,0,0,0,0,0
1,2,0,1,0,2,1991,12.5,2015,0,0,52,3,1,0,0,0,0,0
2,1,0,1,1,2,1987,7.5,2012,0,0,42,20,1,0,0,0,0,0
3,1,0,1,0,3,1982,2.5,2009,0,0,42,26,1,2,0,0,1,0
4,4,0,1,1,3,1990,7.5,2012,0,0,77,2,1,0,0,0,1,0


In [38]:
df1.corr()['Promoted_or_Not'].sort_values(ascending=False)

,Promoted_or_Not
Promoted_or_Not,1.000000
Targets_met,0.224518
Previous_Award,0.201434
Training_score_average,0.178448
Last_performance_score,0.119690
Year_of_birth,0.017991
Division,0.015582
Year_of_recruitment,0.012287
Channel_of_Recruitment,0.006324
State_Of_Origin,0.005488


In [39]:
features = ['Training_score_average', 'Division', 'Targets_met', 'Previous_Award', 'Last_performance_score']
X = df1[features].copy()
y = df1['Promoted_or_Not']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 42)

In [43]:

gb = GradientBoostingClassifier(random_state=42)

# 2. Define the 'knobs' to tune (Parameter names are slightly different)
param_dist = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],        # equivalent to n_estimators
    'max_depth': [3, 5, 7]
}

# 3. Run the Search
random_search = RandomizedSearchCV(
    gb,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42
)

promotion_model = random_search.fit(X_train, y_train)


In [45]:
def calculate_scores(y_true, y_pred, label):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return {
        'Model': label,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# to test if the model is underfitting or overfitting
train_preds = promotion_model.predict(X_train)
test_preds = promotion_model.predict(X_test)

# Calculate scores for both
comparison = pd.DataFrame([
    calculate_scores(y_train, train_preds, "Train (Practice)"),
    calculate_scores(y_test, test_preds, "Test (Exam)")
])

print(comparison)

              Model  Accuracy  Precision    Recall  F1 Score
0  Train (Practice)  0.946817   0.978702  0.374903  0.542135
1       Test (Exam)  0.937100   0.893617  0.314843  0.465632


In [46]:
# 4. Predict and Check Results
y_pred = random_search.predict(X_test)
print(pd.DataFrame([calculate_scores(y_test, y_pred, "Balanced Model")]))

            Model  Accuracy  Precision    Recall  F1 Score
0  Balanced Model    0.9371   0.893617  0.314843  0.465632


In [47]:
# 1. Ask the NEW Random Search model to make predictions
# (It automatically uses the best version it found)
new_weighted_preds = random_search.predict(X_test)

# 2. Calculate the scores for this new attempt
new_results = calculate_scores(y_test, new_weighted_preds, "Weighted Random Search")

# 3. Display the result
print(pd.DataFrame([new_results]))

                    Model  Accuracy  Precision    Recall  F1 Score
0  Weighted Random Search    0.9371   0.893617  0.314843  0.465632
